<a href="https://colab.research.google.com/github/JackGraymer/Advanced-GenAI/blob/main/3_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Generative Artificial Intelligence
**Project - Designing a RAG-Based Q&A System for News Retrieval**

**Authors:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan (Group 5)


# Step 3 Evaluation – Assessing answer quality through both automated and human evaluation

**Contribution:** Vsevolod Mironov, Pascal Küng, Alvaro Cervan

**Goal of this step:** Students will assess the quality of answers produced by their top-performing RAG pipeline. This involves applying the pipeline to benchmark questions, comparing its responses to ground truth answers, and evaluating performance using both automated metrics and human judgment.

### **Objective**
Evaluate the quality of answers generated by the best RAG pipeline using both automated metrics and human judgment.

---

### **Workflow**

1. **Run the Best RAG Pipeline**
	- Apply the developed RAG pipeline to the benchmark question set to generate answers.

2. **Automated Metrics Calculation**
	- **Semantic Exact Match:** Assess if generated answers semantically match ground truth using embeddings or semantic models.
	- **Semantic F1 Score:** Tokenize answers and compute precision, recall, and F1 based on semantic similarity.
	- **BLEU/ROUGE:** Measure N-gram or sequence overlap between generated and ground truth answers.
	- **Record Results:** Store all metric scores in a structured format for comparison.

3. **Human Evaluation**
	- **Criteria:** Evaluate each answer for:
	  - *Relevance* (alignment with the query)
	  - *Correctness* (accuracy vs. ground truth)
	  - *Clarity* (understandability)
	- **Rating Scale:** Use a 1–5 scale for each criterion.
	- **Analysis:** Provide a brief written summary of human evaluation findings.

4. **Report and Presentation**
	- Present both automated and human evaluation results using tables, charts, or other concise visualizations within the notebook.

#### Visual Representation of the Evaluation Pipeline
![Pipeline Overview](Evaluation_Workflow.svg)

---

**Summary Table Example:**

| Question | Automated Metrics (F1, BLEU, ROUGE) | Human Relevance | Human Correctness | Human Clarity | Comments |
|----------|-------------------------------------|-----------------|-------------------|--------------|----------|
| Q1       | ...                                 | ...             | ...               | ...          | ...      |



# 1.0 Loading Data and functions from previous stage

## 1.1 Setup of the environment

In [ ]:
import os
import re
import json
import asyncio
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import pickle
import faiss
from typing import Optional, List
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from FlagEmbedding import FlagReranker
from google.colab import userdata

In [ ]:
# Set the seed for consistent results
seed_value = 2138247234
random.seed(seed_value)
np.random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
base_folder = '/content/drive/MyDrive/AdvGenAI'

In [ ]:
# Run this cell if working locally
df = pd.read_csv('data/Stage2-02-chunked-dataset.csv')
with open('data/Stage2-08-q-a-file-with-relevancy.pkl', 'rb') as f:
	Q_A_ground_thruth_relevancy_dict = pickle.load(f)

	filename = 'data/Stage3-01-precalc-retrieved-chunks.pkl'
	with open(filename, 'rb') as f:
		precalc_retrieved_chunks = pickle.load(f)
	print(f"Dictionary loaded from {filename}:")

# load files in local computer and api from data .env
from dotenv import dotenv_values

env_vars = dotenv_values('data/.env')
OPENAI_API_KEY = env_vars.get('OPENAI_API_KEY', None)
print(f"Loaded OPENAI_API_KEY: {'***' if OPENAI_API_KEY else 'Not found'}")